In [5]:
import os
import openai
import time
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report

openai.api_key = os.getenv("OPENAI_API_KEY")

import json
import numpy as np

np.random.seed(0)
features_valid = json.load(open("../data/valid_subset_text.json"))
features_valid_erp = json.load(open("../dataset_bias/valid_subset_text_erp.json"))
features_valid_tense_all = json.load(open("../dataset_bias/valid_subset_text_tense_bias_vague.json"))

# features_valid_tense_vague = [item for item in features_valid_tense_all if item['labels'][0] == 3]
# features_valid_dep = json.load(open("../dataset_bias/valid_text_features_matres_dep_bias.json"))

In [11]:
features_valid_gpt2xl = json.load(open("../data/valid_subset_text_gpt2xl_bias.json"))
gpt2xl_subset_idx_hard = np.load("../dataset_bias/gpt2xl_subset_idx_hard.npy", allow_pickle=True)
features_valid_gpt2xl_hard = [features_valid[i] for i in gpt2xl_subset_idx_hard]

In [16]:
features_valid[0]

{'text': "[CLS] With a tense Miami waiting, federal immigration officials said Monday they will order the Miami relatives of 6-year-old Cuban rafter Elian Gonzalez to turn the boy over to his father, who could come to the U.S. from Cuba as early as Tuesday.[SEP] ``Our goal is to reunite Elian and his father,'' said Maria Cardona, a spokeswoman for the U.S. Immigration and Naturalization Service.[SEP] ``The issue is not whether we will transfer Elian to his father, but when and how.''[SEP]",
 'e1': 'waiting',
 'e2': 'said',
 'labels': [0]}

# 1. zero_shot

"Determine the temporal order from \"{e1}\" to \"{e2}\" in the following sentence: \"{context}\". Only answer one word from AFTER, BEFORE, EQUAL, VAGUE. Answer:"

In [ ]:
convert_dict = {'BEFORE':0, 'AFTER':1, 'EQUAL':2, 'VAGUE':3}
def parse_result(ans):
    return convert_dict[ans]


In [30]:
zs_results = []
for i in tqdm(range(len(features_valid))):
    item = features_valid[i]
    context = item['text'].replace("[CLS]", "").replace("[SEP]", "").strip()
    e1 = item['e1']
    e2 = item['e2']
    
    prompt = f"Determine the temporal order from \"{e1}\" to \"{e2}\" in the following sentence: \"{context}\". Only answer one word from AFTER, BEFORE, EQUAL, VAGUE. Answer:"
#     print(prompt)
    while True:
        try:
            zs_results.append(openai.Completion.create(
                        model="text-davinci-003",
                        prompt=prompt,
                        max_tokens=20,
                        temperature=0
            ))
            break
        except:
            time.sleep(10)
    time.sleep(2)
#     break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [57:34<00:00,  3.45s/it]


In [34]:
ze_preds = [parse_result(item['choices'][0]['text'].strip()) for item in zs_results]

In [31]:
from collections import Counter
Counter([item['choices'][0]['text'].strip() for item in zs_results])

Counter({'BEFORE': 998, 'EQUAL': 2})

In [35]:
labels = [features_valid[i]['labels'][0] for i in range(len(features_valid))]

print(f1_score(labels, ze_preds, average='macro'), f1_score(labels, ze_preds, average="micro"))

0.17126890203813283 0.521


In [37]:
with open("results/subset_chan_prompt_zs.json", "w") as writer:
    json.dump(ze_preds, writer)


# ICL, m way, k-shot